In [ ]:
import pandas as pd
import numpy as np
import copy
import shap
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
from optuna.samplers import TPESampler
from optuna.trial import Trial
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, LeaveOneOut, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn.base import clone
from sklearn import preprocessing

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
filename = './머신러닝데이터.xlsx'

In [ ]:
# Imputation한 데이터를 불러온다
scoring_transition_1 = pd.read_excel(filename, sheet_name='scoring_result_merged', engine='openpyxl')
scoring_transition_0 = pd.read_excel(filename, sheet_name='scoring_origin', engine='openpyxl')
task_completion_time = pd.read_excel(filename, sheet_name='task_completion_time_mean_imput', engine='openpyxl')
sms_reply = pd.read_excel(filename, sheet_name='SMS_reply', engine='openpyxl')
weather_searching_site_region = pd.read_excel(filename, sheet_name='weather_searching_사이트주소', engine='openpyxl')

# 모든 Feature List
sms_reply, weather_searching_사이트주소는 feature 리스트가 같습니다

In [ ]:
# scoring = ['Routine_Screen_Unlock_Pattern', 'Routine_Phone_Register', 'Routine_Phone_Receive', 'Routine_SMS_Reply', 'Routine_Camera', 
# 'Routine_Location_Searching', 'Routine_Weather_Searching', 'Routine_Transfer', 'Routine_Location_Switching', 'Routine_Weather_Switching', 
# 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 'Transition_Camera', 'Transition_Location_Searching', 
# 'Transition_Weather_Searching', 'Transition_Transfer', 'Transition_Location_Switching', 'Transition_Weather_Switching', 'Result_Phone_Register', 
# 'Result_Phone_Receive', 'Result_SMS_Reply', 'Result_Camera', 'Result_Location_Searching', 'Result_Weather_Searching', 'Result_Transfer', 
# 'Result_Location_Switching', 'Result_Weather_Switching', 'routine_sum_trial', 'transition_sum_trial', 'result_sum_trial', 'all_sum_trial']

# task = ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_total_time_nt', 
# 'phone_register_screen_unlocking_time', 'phone_register_sms_start_time', 'phone_register_instruction_check_time', 'phone_register_total_time', 
# 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_receive_noti response', 'phone_receive_total_time_nt', 'phone_receive_total_time', 
# 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_noti_response_time', 'sms_reply_total_time_nt', 'sms_reply_compeltion_time', 
# 'sms_reply_total_time', 'camera_noti_response', 'camera_total_time_nt', 'camera_instruction_check_time', 'camera_total_time', 'camera_taken_time', 
# 'camera_gallery_delete_time', 'transfer_noti_response_time', 'transfer_total_time_nt', 'transfer_instruction_check_time', 'transfer_total_time', 
# 'transfer_usage_time', 'transfer_share_time', 'weather_searching_noti_response', 'weather_searching_total_time_nt', 'weather_searching_instruction_check_time', 
# 'weather_searching_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_switching_notification_response_time', 
# 'weather_switching_total_time_nt', 'weather_switching_instruction_check_time', 'weather_switching_total_time', 
# 'weather_switching_first_searching_keyword_typing_time', 'weather_switching_second_searching_keyword_typing_time', 
# 'weather_switching_information_searching_time', 'weather_switching_information_sharing_texting_time', 'location_searching_noti_response_time', 
# 'location_searching_total_time_nt', 'location_searching_instruction_check_time', 'location_searching_total_time', 'location_searching_food_typing_time', 
# 'location_searching_foodtyping_mapfinding_time', 'location_searching_mapfinding_time', 'location_searching_route_sharing_texting_time', 
# 'location_switching_notification_response', 'location_switching_total_time_nt', 'location_switching_instruction_check', 'location_switching_total_time', 
# 'location_switching_first_map_typing_time', 'location_switching_second_map_typing_time', 'location_switching_map_finding_time', 
# 'location_switching_map_route_sharing_time']

# sms_and_ws = ['average intercharacter time', 'std intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
# 'total_time (S)', 'total_count', 'IS (C+IF+INF+F)', 'backspace (F)', 'C', 'IF', 'INF', 'Shift', 'ISø (IS+S)', 'T (total_count-F)', 'WPS', 'WPM', 'AdjWPS', 
# 'AdjWPM', 'CPS', 'KSPS', 'GPS', 'KSPC', 'GPC', 'MSD', 'COER', 'UER', 'TER', 'CE', 'PC', 'UB', 'WB', 'CPC']

In [ ]:
# BAC column만 가져온다(Label)
y = scoring_transition_1['BAC']

# Participant, BAC column을 제거한다
scoring_transition_1 = scoring_transition_1.drop(['Participant', 'BAC'], axis=1)
scoring_transition_0 = scoring_transition_0.drop(['Participant', 'BAC'], axis=1)
task_completion_time = task_completion_time.drop(['Participant', 'BAC'], axis=1)
sms_reply = sms_reply.drop(['Participant', 'BAC'], axis=1)
weather_searching_site_region = weather_searching_site_region.drop(['Participant', 'BAC'], axis=1)

In [ ]:
# 라벨 3개용 features
# 각 데이터별로 필요한 column들만 뽑는다
scoring_transition_1_columns = ['Routine_Screen_Unlock_Pattern', 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 
'Transition_Camera', 'Transition_Weather_Searching', 'Transition_Transfer', 'transition_sum_trial']

scoring_transition_0_columns = []

task_completion_time_columns = ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 
'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 
'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 
'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 
'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time']

sms_reply_columns = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'GPS', 'COER', 'UER', 'TER', 'UB', 'WB']

weather_searching_site_region_columns = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'COER', 'UB', 'WB']

In [ ]:
# 라벨을 2개용 features
scoring_transition_1_columns_two_label = ['Routine_Screen_Unlock_Pattern', 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 
'Transition_Camera', 'Transition_Weather_Searching', 'Transition_Transfer', 'transition_sum_trial']

scoring_transition_0_columns_two_label = []

task_completion_time_columns_two_label = ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 
'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 
'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 
'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 
'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time']

sms_reply_columns_two_label = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'GPS', 'COER', 'UER', 'TER', 'UB', 'WB']

weather_searching_site_region_columns_two_label = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'COER', 'UB', 'WB']

In [ ]:
# 위에서 정한 feature들로 각각의 데이터 프레임을 생성한다
scoring_transition_1_df = scoring_transition_1[scoring_transition_1_columns]
scoring_transition_0_df = scoring_transition_0[scoring_transition_0_columns]
task_completion_time_df = task_completion_time[task_completion_time_columns]
sms_reply_df = sms_reply[sms_reply_columns]
weather_searching_site_region_df = weather_searching_site_region[weather_searching_site_region_columns]

scoring_transition_1_df_two_label = scoring_transition_1[scoring_transition_1_columns_two_label]
scoring_transition_0_df_two_label = scoring_transition_0[scoring_transition_0_columns_two_label]
task_completion_time_df_two_label = task_completion_time[task_completion_time_columns_two_label]
sms_reply_df_two_label = sms_reply[sms_reply_columns_two_label]
weather_searching_site_region_df_two_label = weather_searching_site_region[weather_searching_site_region_columns_two_label]

# feature명 수정하기
위에서 적은 feature명을 그대로 복사해서 가져온 다음 바꾸려는 feature만 수정하면 됩니다

In [ ]:
# 라벨 3개용 features의 feature명을 수정한다!
scoring_transition_1_columns = ['Routine_Screen_Unlock_Pattern', 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 
'Transition_Camera', 'Transition_Weather_Searching', 'Transition_Transfer', 'transition_sum_trial']

scoring_transition_0_columns = []

task_completion_time_columns = ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 
'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 
'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 
'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 
'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time']

sms_reply_columns = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'GPS', 'COER', 'UER', 'TER', 'UB', 'WB']

weather_searching_site_region_columns = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'COER', 'UB', 'WB']

scoring_transition_1_df.columns = scoring_transition_1_columns
scoring_transition_0_df.columns = scoring_transition_0_columns
task_completion_time_df.columns = task_completion_time_columns
sms_reply_df.columns = sms_reply_columns
weather_searching_site_region_df.columns = weather_searching_site_region_columns

In [ ]:
# 라벨을 2개용 features의 feature명을 수정한다!
scoring_transition_1_columns_two_label = ['Routine_Screen_Unlock_Pattern', 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 
'Transition_Camera', 'Transition_Weather_Searching', 'Transition_Transfer', 'transition_sum_trial']

scoring_transition_0_columns_two_label = []

task_completion_time_columns_two_label = ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 
'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 
'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 
'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 
'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time']

sms_reply_columns_two_label = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'GPS', 'COER', 'UER', 'TER', 'UB', 'WB']

weather_searching_site_region_columns_two_label = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'COER', 'UB', 'WB']

scoring_transition_1_df_two_label.columns = scoring_transition_1_columns_two_label
scoring_transition_0_df_two_label.columns = scoring_transition_0_columns_two_label
task_completion_time_df_two_label.columns = task_completion_time_columns_two_label
sms_reply_df_two_label.columns = sms_reply_columns_two_label
weather_searching_site_region_df_two_label.columns = weather_searching_site_region_columns_two_label

In [ ]:
# sms reply와 weather searching 사이트 주소의 feature명이 똑같으므로 feature명을 수정한다
sms_reply_df_columns = ['sms_' + col_name for col_name in sms_reply_df.columns] # sms reply feature 이름 앞에 sms_를 붙인다
weather_searching_site_region_df_columns = ['ws_' + col_name for col_name in weather_searching_site_region_df.columns] # ws feature 이름 앞에 ws_를 붙인다

sms_reply_df.columns = sms_reply_df_columns # 수정된 sms reply feature명을 적용한다
weather_searching_site_region_df.columns = weather_searching_site_region_df_columns # 수정된 ws feature명을 적용한다

# 위와 동일하게 작동한다
sms_reply_df_two_label_columns = ['sms_' + col_name for col_name in sms_reply_df_two_label.columns]
weather_searching_site_region_df_two_label_columns = ['ws_' + col_name for col_name in weather_searching_site_region_df_two_label.columns]

sms_reply_df_two_label.columns = sms_reply_df_two_label_columns
weather_searching_site_region_df_two_label.columns = weather_searching_site_region_df_two_label_columns

### merged_df : Label 3개인 데이터 집합
### merged_df_two_label : Label 2개인 데이터 집합

In [ ]:
# 필요한 데이터 프레임들을 합친다
merged_df_list = [scoring_transition_1_df, scoring_transition_0_df, task_completion_time_df, sms_reply_df, weather_searching_site_region_df]
merged_df_list_two_label = [scoring_transition_1_df_two_label, scoring_transition_0_df_two_label, task_completion_time_df_two_label, 
sms_reply_df_two_label, weather_searching_site_region_df_two_label]

merged_df = pd.concat(merged_df_list, axis=1)
merged_df_two_label = pd.concat(merged_df_list_two_label, axis=1)

In [ ]:
# Categorical 데이터에 대해서 인코딩한다
# 어떤 데이터가 Categorical 데이터인가?

# 라벨에 대해서 인코딩한다
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

### N : 3개 라벨에 대한 feature 개수 / M : 2개 라벨에 대한 feature 개수
merged_df.shape = (360, N), y.shape = (360, 1), 라벨 수: 3 </br>
merged_df_two_label.shape = (360, M), changed_label.shape = (360, 1), 라벨 수: 2 </br>
changed_df.shape = (240, M), changed_y.shape = (240, 1), 라벨 수: 2 </br>

In [ ]:
# 라벨에 대해서 인코딩한다(0 -> 0, 0.06 -> 1, 0.09 -> 2)
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

# 3개의 라벨을 2개의 라벨로 바꾼다
changed_label = copy.deepcopy(y)

for i in range(len(changed_label)):
    if changed_label[i] == 1:
        changed_label[i] = 0

for i in range(len(changed_label)):
    if changed_label[i] == 2:
        changed_label[i] = 1

# 라벨 0과 라벨 0.06의 평균을 구해서 하나의 라벨로 만든다
temp_y = pd.Series(y, name='Label')
temp_df = pd.concat([temp_y, merged_df_two_label], axis=1)

changed_df = pd.DataFrame(columns=temp_df.columns)

num = 0
for row_idx in range(0, len(temp_df), 9):
    for i in range(3):
        changed_df.loc[num] = (temp_df.loc[row_idx + i] + temp_df.loc[row_idx + i + 3])/2
        num += 1
    for i in range(3):
        changed_df.loc[num] = temp_df.loc[row_idx + 6 + i]
        num += 1

changed_y = changed_df['Label']
changed_df = changed_df.drop(['Label'], axis=1)
changed_y = encoder.fit_transform(changed_y)

# 3 Labels, 2 Labels(just change), 2 Labels(average)에 대한 데이터
label_processing = ['3 Labels', '2 Labels(just change)', '2 Labels(average)']
data_list = [(merged_df, y), (merged_df_two_label, changed_label), (changed_df, changed_y)]

In [ ]:
scaler_list = {
    'minmax': MinMaxScaler(),
    'maxabs': MaxAbsScaler(),
    'standard': StandardScaler(),
    'robust': RobustScaler()
}

label_list = {
    'three_label': data_list[0],
    'two_label_changed': data_list[1],
    'two_label_average': data_list[2]
}

# scaler_name과 label_name을 인자로 받아서 해당하는 X, y를 반환한다
def getXy(scaler_name, label_name):
    if scaler_name not in scaler_list.keys() and scaler_name != 'no_scale':
        raise ValueError("scaler_name must be 'minmax' or 'maxabs' or 'standard' or 'robust' or 'no_scale!")

    if label_name not in label_list.keys():
        raise ValueError("label_name must be 'three_label' or 'two_label_changed' or 'two_label_average'!")

    X, y = label_list[label_name]

    if scaler_name != 'no_scale':
        scaler = scaler_list[scaler_name]
        df = X.copy()

        df[df.columns] = scaler.fit_transform(df[df.columns])

        X = df
        X[X.columns] = scaler.fit_transform(X[X.columns])
    
    return (X, y)

# 어떤 label의 데이터로 돌릴 것인지, 어떤 scaler를 적용할 것인지, inner loop에서 어떤 cv 방법을 적용할 것인지, Stratified K-fold의 K를 정한다

### scaler_name = 'minmax' or 'maxabs' or 'standard' or 'robust' or 'no_scale'
### label_name = 'three_label' or 'two_label_changed' or 'two_label_average'
### cv_method = 'loso' or 'k_fold'
### n_fold = 39(default)

In [ ]:
scaler_name = 'no_scale'
label_name = 'two_label_changed'
cv_method = 'k_fold'
n_fold = 39

X, y = getXy(scaler_name=scaler_name, label_name=label_name)

# Optuna의 반복 수를 설정한다

In [ ]:
n_trials = 1

In [ ]:
def NestedCVwithOptuna(objective, clf, clf_name):
    metric_df = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc_score'])
    hyper_df_list = []
    feature_df_list = []

    # 클래스가 몇 개인지 확인한다
    class_nums = len(set(y))
    print('class_nums : ', class_nums)
    n_splits = 40
    kfold = KFold(n_splits=n_splits)

    mean_accuracy = 0
    mean_precision = 0
    mean_recall = 0
    mean_f1 = 0
    mean_roc_auc = 0
    
    list_shap_values = list()
    list_test_sets = list()
    
    shap_df_list = [[], [], []]

    for num, (train_idx, val_idx) in enumerate(kfold.split(X)):
        X_train, X_val = X.iloc[train_idx, :], X.iloc[val_idx, :]
        y_train, y_val = y[train_idx], y[val_idx]
        
        classifier = clone(clf)

        # Feature Selection을 진행한다
        selector = RFECV(classifier, step=2, cv=KFold(n_splits=3), min_features_to_select=20)
        selector = selector.fit(X_train, y_train)

        columns = X_train.columns
        selected_list = selector.support_

        selected_features = [col_name for i, col_name in enumerate(columns) if selected_list[i]]

        # 뽑힌 feature들로 데이터를 만든다
        X_train = X_train[selected_features]
        X_val = X_val[selected_features]

        # Bayesian Optimization을 진행한다
        sampler = TPESampler()

        study = optuna.create_study(direction='maximize', sampler=sampler, study_name=f'{clf_name} Study')
        study.optimize(objective(X_train, y_train), n_trials=n_trials)

        best_params = study.best_params
        print('Optuna Best score : ', study.best_value)
        print('Best parameters : ', best_params)
        print('Selected features Num: ', len(selected_features))
        print('Selected features : ', selected_features)

        # Best feature를 저장하는 데이터 프레임을 생성한다
        feature_df = pd.DataFrame([selected_features])
        feature_df_list.append(feature_df)

        # 하이퍼 파라미터를 저장하는 데이터 프레임을 생성한다
        hyper_df = pd.DataFrame([list(best_params.values())], columns=list(best_params.keys()))
        hyper_df_list.append(hyper_df)

        if clf_name == 'lightgbm':
            best_clf = LGBMClassifier(**best_params, random_state=42)
        elif clf_name == 'xgboost':
            best_clf = XGBClassifier(**best_params, random_state=42)
        elif clf_name == 'randomforest':
            best_clf = RandomForestClassifier(**best_params, random_state=42)
        elif clf_name == 'gbm':
            best_clf = GradientBoostingClassifier(**best_params, random_state=42)
        
        best_clf.fit(X_train, y_train)

        y_pred = best_clf.predict(X_val)
        y_pred_prob = best_clf.predict_proba(X_val)

        # 개별 performance metric을 구한다
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred, average='macro')
        recall = recall_score(y_val, y_pred, average='macro')
        f1 = f1_score(y_val, y_pred, average='macro')
        if class_nums > 2:
            roc_auc = roc_auc_score(y_val, y_pred_prob, average='macro', multi_class='ovr')
        else:
            roc_auc = roc_auc_score(y_val, y_pred)
        
        # 개별 performance metric을 데이터 프레임으로 저장한다
        metric_df.loc[len(metric_df)] = [f'model_{num+1}', accuracy, precision, recall, f1, roc_auc]

        # 평균 performance metric을 구한다
        mean_accuracy += accuracy / n_splits
        mean_precision += accuracy / n_splits
        mean_recall += recall / n_splits
        mean_f1 += f1 / n_splits
        mean_roc_auc += roc_auc / n_splits

        print('test accuracy : ', accuracy)

        # explaining model
        explainer = shap.TreeExplainer(best_clf)
        shap_values = explainer.shap_values(X_val)
        
        if class_nums > 2:
            for i in range(3):
                shap_df = pd.DataFrame(shap_values[i], columns=X_train.columns)
                shap_df_list[i].append(shap_df)
        elif len(shap_values) == 2:
            for i in range(2):
                shap_df = pd.DataFrame(shap_values[i], columns=X_train.columns)
                shap_df_list[i].append(shap_df)
        else:
            shap_df = pd.DataFrame(shap_values, columns=X_train.columns)
            shap_df_list[0].append(shap_df)

    print()
    print('mean accuracy : ', mean_accuracy)
    print('mean precision : ', mean_precision)
    print('mean recall : ', mean_recall)
    print('mean f1 : ', mean_f1)
    print('mean roc_auc : ', mean_roc_auc)
    print()

    metric_df.loc[len(metric_df)] = ['mean', mean_accuracy, mean_precision, mean_recall, mean_f1, mean_roc_auc]
    hyper_df_merged = pd.concat(hyper_df_list, ignore_index=True)
    feature_df_merged = pd.concat(feature_df_list, ignore_index=True)

    result_df = pd.concat([metric_df, hyper_df_merged, feature_df_merged], axis=1)

    return (shap_df_list, result_df)

## Accuracy가 아닌 다른 metric을 기준으로 Optuna을 하려면 precision ~ roc_auc 주석 중 하나를 풀고 return 해주면 됩니다

In [ ]:
def InnerCrossValidation(clf, X, y, cv_method, n_fold=39):
    class_nums = len(set(y))
    if cv_method == 'loso':
        kfold = KFold(n_splits=39)
        splits = kfold.split(X)
    elif cv_method == 'k_fold':
        stratified_kfold = StratifiedKFold(n_splits=n_fold)
        splits = stratified_kfold.split(X, y)
    else:
        raise ValueError("cv_method must be 'loso' or 'k_fold'!")
    
    train_X_len = len(X)
    if train_X_len % n_fold != 0:
        raise ValueError(f"{train_X_len} / {n_fold} is not Integer!")

    y_true_list, y_pred_list = [], []
    y_pred_prob_list = []
    
    for train_idx, val_idx in splits:
        X_train, X_val = X.iloc[train_idx, :], X.iloc[val_idx, :]
        y_train, y_val = y[train_idx], y[val_idx]
        
        classifier = clone(clf)

        # 모델을 학습한다
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_val)
        y_pred_prob = classifier.predict_proba(X_val)
        
        # 실제, 예측 라벨을 저장한다
        y_true_list.extend(y_val)
        y_pred_list.extend(y_pred)
        y_pred_prob_list.extend(y_pred_prob)
    
    # 여기를 바꾸면 됩니다
    accuracy = accuracy_score(y_true_list, y_pred_list)
    # precision = precision_score(y_true_list, y_pred_list, average='macro')
    # recall = recall_score(y_true_list, y_pred_list, average='macro')
    # f1 = f1_score(y_true_list, y_pred_list, average='macro')
    # if class_nums > 2:
    #     roc_auc = roc_auc_score(y_val, y_pred_prob, average='macro', multi_class='ovr')
    # else:
    #     roc_auc = roc_auc_score(y_val, y_pred)

    # 반환하는 변수를 바꿔줘야 합니다
    return accuracy

# Lightgbm Bayesian

In [ ]:
class LGBMObjective(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __call__(self, trial: Trial):
        lgbm_params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'learning_rate': trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1), 
            'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 5.0), # lambda_l1
            'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 5.0), # lambda_l2
            'random_state': 42
        }

        clf = LGBMClassifier(**lgbm_params)

        mean_accuracy = InnerCrossValidation(clf, self.X, self.y, cv_method, n_fold)

        return mean_accuracy

In [ ]:
clf = LGBMClassifier(random_state=42)
shap_df_list, metric_df = NestedCVwithOptuna(LGBMObjective, clf, 'lightgbm')

class_nums :  2


[I 2022-08-31 00:56:24,242] A new study created in memory with name: lightgbm Study
[I 2022-08-31 00:56:28,944] Trial 0 finished with value: 0.7663817663817664 and parameters: {'n_estimators': 326, 'learning_rate': 0.041828196885327226, 'num_leaves': 11, 'colsample_bytree': 1.0, 'min_child_samples': 11, 'subsample': 0.6, 'reg_alpha': 4.598137104176723, 'reg_lambda': 3.015806548435523}. Best is trial 0 with value: 0.7663817663817664.


Optuna Best score :  0.7663817663817664
Best parameters :  {'n_estimators': 326, 'learning_rate': 0.041828196885327226, 'num_leaves': 11, 'colsample_bytree': 1.0, 'min_child_samples': 11, 'subsample': 0.6, 'reg_alpha': 4.598137104176723, 'reg_lambda': 3.015806548435523}
Selected features Num:  22
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'camera_gallery_delete_time', 'transfer_usage_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'sms_min intercharacter time', 'sms_max intercharacter time', 'sms_KSPS', 'sms_TER', 'ws_average intercharacter time', 'ws_median intercharacter time', 'ws_CPS', 'ws_COER']
test accuracy :  1.0


[I 2022-08-31 00:56:32,159] A new study created in memory with name: lightgbm Study
[I 2022-08-31 00:56:34,457] Trial 0 finished with value: 0.7692307692307693 and parameters: {'n_estimators': 349, 'learning_rate': 0.0612219574420988, 'num_leaves': 4, 'colsample_bytree': 0.9, 'min_child_samples': 98, 'subsample': 0.9, 'reg_alpha': 4.485657317144603, 'reg_lambda': 4.243347733069335}. Best is trial 0 with value: 0.7692307692307693.


Optuna Best score :  0.7692307692307693
Best parameters :  {'n_estimators': 349, 'learning_rate': 0.0612219574420988, 'num_leaves': 4, 'colsample_bytree': 0.9, 'min_child_samples': 98, 'subsample': 0.9, 'reg_alpha': 4.485657317144603, 'reg_lambda': 4.243347733069335}
Selected features Num:  26
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'sms_median intercharacter time', 'sms_min intercharacter time', 'sms_max intercharacter time', 'sms_CPS', 'sms_KSPS', 'sms_TER', 'ws_median intercharacter time', 'ws_min intercharacter time', 'ws_CPS', '

...

Optuna Best score :  0.7521367521367521
Best parameters :  {'n_estimators': 194, 'learning_rate': 0.008831578251122248, 'num_leaves': 119, 'colsample_bytree': 0.6, 'min_child_samples': 76, 'subsample': 0.8, 'reg_alpha': 3.446032884683929, 'reg_lambda': 3.8385900999435916}
Selected features Num:  42
Selected features :  ['transition_sum_trial', 'average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time', 'sms_aver

[I 2022-08-31 01:00:47,188] A new study created in memory with name: lightgbm Study
[I 2022-08-31 01:00:47,930] Trial 0 finished with value: 0.7464387464387464 and parameters: {'n_estimators': 123, 'learning_rate': 0.007086857514124043, 'num_leaves': 69, 'colsample_bytree': 0.7, 'min_child_samples': 64, 'subsample': 0.8, 'reg_alpha': 3.9606640415638066, 'reg_lambda': 3.7021154006513943}. Best is trial 0 with value: 0.7464387464387464.


Optuna Best score :  0.7464387464387464
Best parameters :  {'n_estimators': 123, 'learning_rate': 0.007086857514124043, 'num_leaves': 69, 'colsample_bytree': 0.7, 'min_child_samples': 64, 'subsample': 0.8, 'reg_alpha': 3.9606640415638066, 'reg_lambda': 3.7021154006513943}
Selected features Num:  42
Selected features :  ['Transition_Camera', 'transition_sum_trial', 'average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_t

[I 2022-08-31 01:00:49,609] A new study created in memory with name: lightgbm Study
[I 2022-08-31 01:00:52,639] Trial 0 finished with value: 0.7806267806267806 and parameters: {'n_estimators': 462, 'learning_rate': 0.006713100691044636, 'num_leaves': 142, 'colsample_bytree': 0.6, 'min_child_samples': 43, 'subsample': 1.0, 'reg_alpha': 3.909680680593169, 'reg_lambda': 4.40560655815576}. Best is trial 0 with value: 0.7806267806267806.


Optuna Best score :  0.7806267806267806
Best parameters :  {'n_estimators': 462, 'learning_rate': 0.006713100691044636, 'num_leaves': 142, 'colsample_bytree': 0.6, 'min_child_samples': 43, 'subsample': 1.0, 'reg_alpha': 3.909680680593169, 'reg_lambda': 4.40560655815576}
Selected features Num:  36
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time', 'sms_average intercharacter time', 'sms_median intercharacter time', 'sm

## 40개의 모델의 개별 performance metric + 평균 performnace metric을 저장한다
## 각 모델별 test set의 feature에 따른 shap value를 저장한다

In [ ]:
if label_name == 'three_label':
    shap_values_df_0 = pd.concat(shap_df_list[0])
    shap_values_df_1 = pd.concat(shap_df_list[1])
    shap_values_df_2 = pd.concat(shap_df_list[2])

    with pd.ExcelWriter(f"./lgbm_result.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df_0.to_excel(writer, sheet_name="shap_list_0", index=False)
        shap_values_df_1.to_excel(writer, sheet_name="shap_list_1", index=False)
        shap_values_df_2.to_excel(writer, sheet_name="shap_list_2", index=False)

if label_name != 'three_label':
    shap_values_df = pd.concat(shap_df_list[0])

    with pd.ExcelWriter(f"./lgbm_result.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df.to_excel(writer, sheet_name="shap_list", index=False)

# Xgboost Bayesian

In [ ]:
class XBGObjective(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __call__(self, trial: Trial):    
        xgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'learning_rate': trial.suggest_loguniform("learning_rate", 1e-4, 1e-1),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
            'gamma': trial.suggest_float('gamma', 0.0, 5.0, step=0.1),
            'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 5.0), # lambda_l1
            'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 5.0), # lambda_l2
            'random_state': 42
        }

        clf = XGBClassifier(**xgb_params)

        mean_accuracy = InnerCrossValidation(clf, self.X, self.y, cv_method, n_fold)

        return mean_accuracy

In [ ]:
clf = XGBClassifier(random_state=42)
shap_df_list, metric_df = NestedCVwithOptuna(XBGObjective, clf, 'xgboost')

class_nums :  2


[I 2022-08-31 00:40:22,751] A new study created in memory with name: xgboost Study
[I 2022-08-31 00:40:30,160] Trial 0 finished with value: 0.7834757834757835 and parameters: {'n_estimators': 479, 'learning_rate': 0.02027461225419885, 'max_depth': 5, 'min_child_weight': 4, 'colsample_bytree': 1.0, 'subsample': 0.5, 'gamma': 1.7000000000000002, 'reg_alpha': 3.2839562817091927, 'reg_lambda': 4.132672346068551}. Best is trial 0 with value: 0.7834757834757835.


Optuna Best score :  0.7834757834757835
Best parameters :  {'n_estimators': 479, 'learning_rate': 0.02027461225419885, 'max_depth': 5, 'min_child_weight': 4, 'colsample_bytree': 1.0, 'subsample': 0.5, 'gamma': 1.7000000000000002, 'reg_alpha': 3.2839562817091927, 'reg_lambda': 4.132672346068551}
Selected features Num:  28
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'camera_taken_time', 'camera_gallery_delete_time', 'transfer_usage_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'sms_average intercharacter time', 'sms_min intercharacter time', 'sms_max intercharacter time', 'sms_CPS', 'sms_KSPS', 'sms_GPS', 'sms_COER', 'sms_TER', 'sms_UB', 'ws_median intercharacter time', 'ws_min

[I 2022-08-31 00:40:41,357] A new study created in memory with name: xgboost Study
[I 2022-08-31 00:40:49,836] Trial 0 finished with value: 0.7578347578347578 and parameters: {'n_estimators': 409, 'learning_rate': 0.0003339333528905027, 'max_depth': 7, 'min_child_weight': 4, 'colsample_bytree': 0.9, 'subsample': 0.7, 'gamma': 3.2, 'reg_alpha': 2.5002626588819394, 'reg_lambda': 4.506800261541564}. Best is trial 0 with value: 0.7578347578347578.


Optuna Best score :  0.7578347578347578
Best parameters :  {'n_estimators': 409, 'learning_rate': 0.0003339333528905027, 'max_depth': 7, 'min_child_weight': 4, 'colsample_bytree': 0.9, 'subsample': 0.7, 'gamma': 3.2, 'reg_alpha': 2.5002626588819394, 'reg_lambda': 4.506800261541564}
Selected features Num:  38
Selected features :  ['Routine_Screen_Unlock_Pattern', 'average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'sms_average interchar

...

[I 2022-08-31 00:44:10,261] A new study created in memory with name: xgboost Study
[I 2022-08-31 00:44:11,609] Trial 0 finished with value: 0.7777777777777778 and parameters: {'n_estimators': 106, 'learning_rate': 0.07655663354173123, 'max_depth': 6, 'min_child_weight': 9, 'colsample_bytree': 0.6, 'subsample': 0.5, 'gamma': 0.2, 'reg_alpha': 3.8339687195753274, 'reg_lambda': 3.1255667678312573}. Best is trial 0 with value: 0.7777777777777778.


Optuna Best score :  0.7777777777777778
Best parameters :  {'n_estimators': 106, 'learning_rate': 0.07655663354173123, 'max_depth': 6, 'min_child_weight': 9, 'colsample_bytree': 0.6, 'subsample': 0.5, 'gamma': 0.2, 'reg_alpha': 3.8339687195753274, 'reg_lambda': 3.1255667678312573}
Selected features Num:  20
Selected features :  ['average_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'camera_taken_time', 'camera_gallery_delete_time', 'transfer_total_time', 'weather_information_searching_time', 'sms_average intercharacter time', 'sms_min intercharacter time', 'sms_CPS', 'sms_KSPS', 'sms_GPS', 'sms_COER', 'sms_TER', 'ws_median intercharacter time', 'ws_CPS', 'ws_KSPS', 'ws_COER']
test accuracy :  0.8888888888888888


[I 2022-08-31 00:44:22,839] A new study created in memory with name: xgboost Study
[I 2022-08-31 00:44:26,107] Trial 0 finished with value: 0.8205128205128205 and parameters: {'n_estimators': 111, 'learning_rate': 0.02681787493968089, 'max_depth': 9, 'min_child_weight': 4, 'colsample_bytree': 1.0, 'subsample': 1.0, 'gamma': 2.4000000000000004, 'reg_alpha': 3.7160525187400353, 'reg_lambda': 1.5027077252048167}. Best is trial 0 with value: 0.8205128205128205.


Optuna Best score :  0.8205128205128205
Best parameters :  {'n_estimators': 111, 'learning_rate': 0.02681787493968089, 'max_depth': 9, 'min_child_weight': 4, 'colsample_bytree': 1.0, 'subsample': 1.0, 'gamma': 2.4000000000000004, 'reg_alpha': 3.7160525187400353, 'reg_lambda': 1.5027077252048167}
Selected features Num:  42
Selected features :  ['Transition_Camera', 'Transition_Weather_Searching', 'transition_sum_trial', 'average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 

KeyboardInterrupt: 

## 40개의 모델의 개별 performance metric + 평균 performnace metric을 저장한다
## 각 모델별 test set의 feature에 따른 shap value를 저장한다

In [ ]:
if label_name == 'three_label':
    shap_values_df_0 = pd.concat(shap_df_list[0])
    shap_values_df_1 = pd.concat(shap_df_list[1])
    shap_values_df_2 = pd.concat(shap_df_list[2])

    with pd.ExcelWriter(f"./xgb_result.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df_0.to_excel(writer, sheet_name="shap_list_0", index=False)
        shap_values_df_1.to_excel(writer, sheet_name="shap_list_1", index=False)
        shap_values_df_2.to_excel(writer, sheet_name="shap_list_2", index=False)

if label_name != 'three_label':
    shap_values_df = pd.concat(shap_df_list[0])

    with pd.ExcelWriter(f"./xgb_result.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df.to_excel(writer, sheet_name="shap_list", index=False)

# Random Forest Bayesian

In [ ]:
class RFObjective(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __call__(self, trial: Trial):
        rf_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'random_state': 42
        }

        clf = RandomForestClassifier(**rf_params)

        mean_accuracy = InnerCrossValidation(clf, self.X, self.y, cv_method, n_fold)

        return mean_accuracy

In [ ]:
clf = RandomForestClassifier(random_state=42)
shap_df_list, metric_df = NestedCVwithOptuna(RFObjective, clf, 'randomforest')

class_nums :  2


[I 2022-08-29 16:01:26,468] A new study created in memory with name: randomforest Study
[I 2022-08-29 16:01:26,815] Trial 0 finished with value: 0.7863247863247863 and parameters: {'n_estimators': 56, 'max_depth': 9, 'min_samples_leaf': 8, 'min_samples_split': 10}. Best is trial 0 with value: 0.7863247863247863.


Optuna Best score :  0.7863247863247863
Best parameters :  {'n_estimators': 56, 'max_depth': 9, 'min_samples_leaf': 8, 'min_samples_split': 10}
Selected features Num:  28
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_total_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time', 'sms_max intercharacter time', 'sms_CPS', 'sms_KSPS', 'sms_COER', 'sms_TER', 'sms_UB', 'sms_WB', 'ws_average intercharacter time', 'ws_median intercharacter time', 'ws_min intercharacter time', 'ws_max intercharacter time', 'ws_CPS', 'ws_KSPS', 'ws_COER', 'ws_UB', 'ws_WB']
test accuracy :  1.0


[I 2022-08-29 16:01:39,150] A new study created in memory with name: randomforest Study
[I 2022-08-29 16:01:41,278] Trial 0 finished with value: 0.792022792022792 and parameters: {'n_estimators': 357, 'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 5}. Best is trial 0 with value: 0.792022792022792.


Optuna Best score :  0.792022792022792
Best parameters :  {'n_estimators': 357, 'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 5}
Selected features Num:  26
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_total_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time', 'sms_average intercharacter time', 'sms_min intercharacter time', 'sms_max intercharacter time', 'sms_CPS', 'sms_TER', 'sms_UB', 'sms_WB', 'ws_average intercharacter time', 'ws_median intercharacter time', 'ws_max intercharacter time', 'ws_CPS', 'ws_KSPS', 'ws_COER', 'ws_WB']
test accuracy :  1.0


...

Optuna Best score :  0.8062678062678063
Best parameters :  {'n_estimators': 146, 'max_depth': 6, 'min_samples_leaf': 5, 'min_samples_split': 9}
Selected features Num:  32
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_call_time', 'phone_register_total_time', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_gallery_delete_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time', 'sms_min intercharacter time', 'sms_max intercharacter time', 'sms_CPS', 'sms_GPS', 'sms_COER', 'sms_TER', 'sms_UB', 'sms_WB', 'ws_average intercharacter time', 'ws_median intercharacter time', 'ws_min intercharacter time', 'ws_max intercharacter time', 'ws_CPS', 'ws_KSPS', 'ws_COER', 'ws_UB', 'ws_WB']
test accuracy :  1.0


[I 2022-08-29 16:09:53,899] A new study created in memory with name: randomforest Study
[I 2022-08-29 16:09:55,062] Trial 0 finished with value: 0.7834757834757835 and parameters: {'n_estimators': 202, 'max_depth': 8, 'min_samples_leaf': 10, 'min_samples_split': 3}. Best is trial 0 with value: 0.7834757834757835.


Optuna Best score :  0.7834757834757835
Best parameters :  {'n_estimators': 202, 'max_depth': 8, 'min_samples_leaf': 10, 'min_samples_split': 3}
Selected features Num:  34
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_missing_call_time_1', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_gallery_delete_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time', 'sms_average intercharacter time', 'sms_median intercharacter time', 'sms_max intercharacter time', 'sms_CPS', 'sms_GPS', 'sms_COER', 'sms_TER', 'sms_UB', 'sms_WB', 'ws_average intercharacter time', 'ws_median intercharacter time', 'ws_min intercharacter time', 'ws_max intercharacter time',

[I 2022-08-29 16:10:06,891] A new study created in memory with name: randomforest Study
[I 2022-08-29 16:10:07,646] Trial 0 finished with value: 0.7891737891737892 and parameters: {'n_estimators': 118, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 7}. Best is trial 0 with value: 0.7891737891737892.


Optuna Best score :  0.7891737891737892
Best parameters :  {'n_estimators': 118, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 7}
Selected features Num:  36
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_screen_unlocking_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time', 'sms_average intercharacter time', 'sms_min intercharacter time', 'sms_max intercharacter time', 'sms_CPS', 'sms_KSPS', 'sms_COER', 'sms_TER', 'sms_UB', 'sms_WB', 'ws_average intercharacter time', 'ws_median intercharacter time', 'ws_min intercharacter time', 

## 40개의 모델의 개별 performance metric + 평균 performnace metric을 저장한다
## 각 모델별 test set의 feature에 따른 shap value를 저장한다

In [ ]:
if label_name == 'three_label':
    shap_values_df_0 = pd.concat(shap_df_list[0])
    shap_values_df_1 = pd.concat(shap_df_list[1])
    shap_values_df_2 = pd.concat(shap_df_list[2])

    with pd.ExcelWriter(f"./rf_result.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df_0.to_excel(writer, sheet_name="shap_list_0", index=False)
        shap_values_df_1.to_excel(writer, sheet_name="shap_list_1", index=False)
        shap_values_df_2.to_excel(writer, sheet_name="shap_list_2", index=False)

if label_name != 'three_label':
    shap_values_df = pd.concat(shap_df_list[0])

    with pd.ExcelWriter(f"./rf_result.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df.to_excel(writer, sheet_name="shap_list", index=False)

# GBM Bayesian

In [ ]:
class GBMObjective(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __call__(self, trial: Trial):
        gbm_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'learning_rate': trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'max_features': trial.suggest_categorical('max_features', [None, 'sqrt']),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1), 
            'random_state': 42
        }

        clf = GradientBoostingClassifier(**gbm_params)

        mean_accuracy = InnerCrossValidation(clf, self.X, self.y, cv_method, n_fold)

        return mean_accuracy

In [ ]:
clf = GradientBoostingClassifier(random_state=42)
shap_df_list, metric_df = NestedCVwithOptuna(GBMObjective, clf, 'gbm')

class_nums :  2


[I 2022-08-29 16:10:31,684] A new study created in memory with name: gbm Study
[I 2022-08-29 16:10:33,331] Trial 0 finished with value: 0.8034188034188035 and parameters: {'n_estimators': 119, 'learning_rate': 0.05462078952589598, 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 6, 'min_samples_split': 8, 'subsample': 0.8}. Best is trial 0 with value: 0.8034188034188035.


Optuna Best score :  0.8034188034188035
Best parameters :  {'n_estimators': 119, 'learning_rate': 0.05462078952589598, 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 6, 'min_samples_split': 8, 'subsample': 0.8}
Selected features Num:  20
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_screen_unlocking_time', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_gallery_delete_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'sms_min intercharacter time', 'sms_CPS', 'sms_KSPS', 'sms_TER', 'sms_UB', 'ws_median intercharacter time', 'ws_CPS', 'ws_KSPS', 'ws_COER', 'ws_UB']
test accuracy :  1.0


[I 2022-08-29 16:10:56,707] A new study created in memory with name: gbm Study
[I 2022-08-29 16:10:56,919] Trial 0 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 56, 'learning_rate': 0.0024899161284824747, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'subsample': 0.7}. Best is trial 0 with value: 0.6666666666666666.


Optuna Best score :  0.6666666666666666
Best parameters :  {'n_estimators': 56, 'learning_rate': 0.0024899161284824747, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'subsample': 0.7}
Selected features Num:  24
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'transfer_total_time', 'weather_information_sharing_time', 'sms_min intercharacter time', 'sms_max intercharacter time', 'sms_CPS', 'sms_KSPS', 'sms_TER', 'sms_UB', 'ws_median intercharacter time', 'ws_CPS', 'ws_KSPS', 'ws_UB', 'ws_WB']
test accuracy :  0.6666666666666666


...

Optuna Best score :  0.7891737891737892
Best parameters :  {'n_estimators': 458, 'learning_rate': 0.015315859105493488, 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 9, 'subsample': 0.9}
Selected features Num:  30
Selected features :  ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'transfer_usage_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'sms_median intercharacter time', 'sms_min intercharacter time', 'sms_max intercharacter time', 'sms_CPS', 'sms_KSPS', 'sms_TER', 'sms_UB', 'sms_WB', 'ws_median intercharacter time', 'ws_max intercharacter time', 'ws_CPS', 'ws_KSPS', 'ws_COER', 'ws_U

[I 2022-08-29 16:26:44,507] A new study created in memory with name: gbm Study
[I 2022-08-29 16:26:49,898] Trial 0 finished with value: 0.7777777777777778 and parameters: {'n_estimators': 327, 'learning_rate': 0.007448840683867543, 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 7, 'min_samples_split': 4, 'subsample': 0.6}. Best is trial 0 with value: 0.7777777777777778.


Optuna Best score :  0.7777777777777778
Best parameters :  {'n_estimators': 327, 'learning_rate': 0.007448840683867543, 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 7, 'min_samples_split': 4, 'subsample': 0.6}
Selected features Num:  50
Selected features :  ['Routine_Screen_Unlock_Pattern', 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 'Transition_Camera', 'Transition_Weather_Searching', 'Transition_Transfer', 'transition_sum_trial', 'average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time'

[I 2022-08-29 16:27:12,112] A new study created in memory with name: gbm Study
[I 2022-08-29 16:27:14,687] Trial 0 finished with value: 0.7720797720797721 and parameters: {'n_estimators': 194, 'learning_rate': 0.05800095443074267, 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 9, 'min_samples_split': 7, 'subsample': 0.6}. Best is trial 0 with value: 0.7720797720797721.


Optuna Best score :  0.7720797720797721
Best parameters :  {'n_estimators': 194, 'learning_rate': 0.05800095443074267, 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 9, 'min_samples_split': 7, 'subsample': 0.6}
Selected features Num:  44
Selected features :  ['Transition_Camera', 'Transition_Weather_Searching', 'average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 'phone_receive_total_time', 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time', 'sms_average intercharacter tim

## 40개의 모델의 개별 performance metric + 평균 performnace metric을 저장한다
## 각 모델별 test set의 feature에 따른 shap value를 저장한다

In [ ]:
if label_name == 'three_label':
    shap_values_df_0 = pd.concat(shap_df_list[0])
    shap_values_df_1 = pd.concat(shap_df_list[1])
    shap_values_df_2 = pd.concat(shap_df_list[2])

    with pd.ExcelWriter(f"./gbm_result.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df_0.to_excel(writer, sheet_name="shap_list_0", index=False)
        shap_values_df_1.to_excel(writer, sheet_name="shap_list_1", index=False)
        shap_values_df_2.to_excel(writer, sheet_name="shap_list_2", index=False)

if label_name != 'three_label':
    shap_values_df = pd.concat(shap_df_list[0])

    with pd.ExcelWriter(f"./gbm_result.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df.to_excel(writer, sheet_name="shap_list", index=False)